In [2]:
%%time
import os
import json

import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
import torch
from sagemaker.utils import unique_name_from_base

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = get_execution_role()
prefix = 'wenet2x'
output_path = f"s3://{bucket}/{prefix}"

print("torch.__version__:{}".format(torch.__version__))
print("boto3.__version__:{}".format(boto3.__version__))
print("sagemaker.__version__:{}".format(sagemaker.__version__))
print("bucket:{}".format(bucket))
print("role:{}".format(role))

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.__version__:1.12.1+cpu
boto3.__version__:1.24.84
sagemaker.__version__:2.117.0
bucket:sagemaker-us-east-1-348052051973
role:arn:aws:iam::348052051973:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole
CPU times: user 1.97 s, sys: 1.67 s, total: 3.64 s
Wall time: 2.79 s


In [3]:
%%markdown
Copy the wenet/examples/librispeech/s0/*.sh and wenet/examples/librispeech/s0/local to wenet/ as requested by Sagemaker
Overwrite the wenet/wenet/bin/train.py with the given one
Change the /root/wenet to /opt/ml/input in all data.list files (especially for train_960 and dev)

If you are cloning from Github:
The "Librispeech" in data.list file in Github has the wrong captalization because it's wrong when I upload it. Please change it yourself!

Copy the wenet/examples/librispeech/s0/*.sh and wenet/examples/librispeech/s0/local to wenet/ as requested by Sagemaker
Overwrite the wenet/wenet/bin/train.py with the given one
Change the /root/wenet to /opt/ml/input in all data.list files (especially for train_960 and dev)

If you are cloning from Github:
The "Librispeech" in data.list file in Github has the wrong captalization because it's wrong when I upload it. Please change it yourself!


In [10]:
%%time
instance_type = "ml.p3.2xlarge"
max_run = 432000
checkpoint_s3_uri = f"s3://{bucket}/{prefix}/checkpoints"

hyperparameters = {
    'datadir':'/opt/ml/input/data/training',
    'stage': '4',
    'stop_stage': '5',
    'train_config': 'examples/librispeech/s0/conf/train_conformer.yaml',
    'model_dir': '/opt/ml/model',
    'checkpoint_dir': '/opt/ml/checkpoints',
    'output_dir': '/opt/ml/output',
}

est = PyTorch(
    entry_point="run.sh",
    source_dir="./wenet",
    image_uri = "348052051973.dkr.ecr.us-east-1.amazonaws.com/sagemaker-pytorch110:1",
    # framework_version="1.11.0",
    py_version="py38",
    role=role,
    instance_count=1,
    instance_type=instance_type,
    volume_size=200,
    disable_profiler=True,
    debugger_hook_config=False,
    base_job_name=prefix,
    hyperparameters = hyperparameters,
    checkpoint_s3_uri = checkpoint_s3_uri,
    # keep_alive_period_in_seconds=1800,
    max_run = max_run,
    tags = [{"Key": "team", "Value": "asr"}, {"Key": "person", "Value": "andrew"}, {"Key": "project", "Value": "abc"}],
)


CPU times: user 136 ms, sys: 7.87 ms, total: 144 ms
Wall time: 144 ms


In [11]:
from sagemaker.inputs import TrainingInput
prefix_dataset = "wenet/export"
loc =f"s3://{bucket}/{prefix_dataset}"
print(loc)
training = TrainingInput(
    s3_data_type='S3Prefix', # Available Options: S3Prefix | ManifestFile | AugmentedManifestFile
    s3_data=loc,
    distribution='FullyReplicated', # Available Options: FullyReplicated | ShardedByS3Key 
    input_mode='FastFile'
)



s3://sagemaker-us-east-1-348052051973/wenet/export


In [12]:
%%time
job_name = est.fit({"training":training})
#job_name = est.fit()

2022-11-30 03:52:48 Starting - Starting the training job......
2022-11-30 03:53:31 Starting - Preparing the instances for training......
2022-11-30 03:54:44 Downloading - Downloading input data......
2022-11-30 03:55:34 Training - Downloading the training image........................
2022-11-30 03:59:46 Uploading - Uploading generated training model/opt/conda/bin/train: line 3: import: command not found
/opt/conda/bin/train: line 4: import: command not found
/opt/conda/bin/train: line 5: from: command not found
/opt/conda/bin/train: train: line 7: syntax error near unexpected token `('
/opt/conda/bin/train: train: line 7: `    sys.argv[0] = re.sub(r'(-script\.pyw|\.exe)?$', '', sys.argv[0])'

2022-11-30 03:59:57 Failed - Training job failed


UnexpectedStatusException: Error for Training job wenet2x-2022-11-30-03-51-00-537: Failed. Reason: AlgorithmError: , exit code: 2

In [ ]:
model_data = est.model_data
print("Model artifact saved at:\n", model_data)